# RNA MULTICAPA DE 3 CAPAS
 - capa de entrada(N)
 - 1 capa oculta(M)
 - capa de salida(O)

## importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## funcion de activacion

In [ ]:
def func_activacion(n):
  return 1/(1+math.exp(-n))   

In [ ]:
func_activacion_vect = np.vectorize(func_activacion)

## RNA

In [ ]:
def neurona(pesos,entrada, bias):
  prod_punto = np.dot(pesos,entrada)+bias # z = W.X + b
  return func_activacion_vect(prod_punto) # f(z)

## RNA Multicapa

In [ ]:
def red_neuronal_multicapa(input, pesosc1, biasc1, pesoscs, biascs):
  # Capa entrada 
  entrada_x = input
  # Capa oculta 1
  salida_c1 = neurona(pesosc1, entrada_x, biasc1)
  # Capa de SALIDA
  output_cs = neurona(pesoscs, salida_c1, biascs)
  # Retornar resultados  
  return salida_c1, output_cs

## derivada de la funcion

In [ ]:
def derivada_f(salida_y): # derivada de la funcion sigmoide
  return salida_y*(1-salida_y)

In [ ]:
def adjust_output(output_wish,size,ExistZero=False):
  SD = np.zeros(size) # vector con el nro de neuronas 
  if ExistZero == True:
    SD[output_wish] = 1
  else:
    SD[output_wish - 1] = 1
  return SD

## backpropagation

In [ ]:
def propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error=0.005,epochs=1000):
  flag = True if 0 in salidas_deseadas.unique() else False
  # longitud de la entrada(caracteristicas)
  n_entradas = entradas.shape[1]
  # Capa oculta 1
  pesosc1 = 2*np.random.rand(neuronas_c1,n_entradas) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biasc1 = 2*np.random.rand(neuronas_c1)-1
  # Capa de salida
  pesoscs = 2*np.random.rand(neuronas_cs,neuronas_c1) - 1 
  biascs = 2*np.random.rand(neuronas_cs) - 1
  # Iterar el algoritmo(nro de epocas)
  for epoch in range(0,epochs):
    for entrada_i,salida_deseada_i in zip(entradas,salidas_deseadas):
      salida_c1, output_cs = red_neuronal_multicapa(entrada_i, pesosc1, biasc1, pesoscs, biascs)
      # ----------------------------------------------------------------------------------------------------
      # Calcular los errores producidos en cada capa de forma invertida(retropropagacion) y actualizar pesos
      # ----------------------------------------------------------------------------------------------------
      salida_deseada = adjust_output(salida_deseada_i,neuronas_cs,flag)
      # Calcular Δ para la CAPA de SALIDA(cs): 
      delta_cs = derivada_f(output_cs)*(salida_deseada - output_cs)
      peso_anterior_cs = pesoscs
      # Actualizar pesos que llegan a la capa de SALIDA
      pesoscs = pesoscs + alfa * delta_cs.reshape(1,-1).T * salida_c1.reshape(1,-1) 
      biascs = biascs + alfa * delta_cs*1   
      # Calcular Δ para la CAPA OCULTA 1 (pesosc1):      
      delta_c1 = derivada_f(salida_c1) * np.dot(peso_anterior_cs.T, delta_cs) 
      # Actualizar pesos  que entran a las neuronas de la capa oculta 1
      pesosc1 = pesosc1 + alfa * (delta_c1.reshape(1,-1)*entrada_i.reshape(1,-1).T).T 
      biasc1 = biasc1 + alfa * delta_c1 * 1  

  return pesosc1,biasc1,pesoscs,biascs 

## fit RNA

In [ ]:
def fit_rna(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs):
  pesosc1,biasc1,pesoscs,biascs = propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs)
  return pesosc1,biasc1,pesoscs,biascs

## predict RNA

In [ ]:
def predict_rna(x_test, valores_obtenidos):
  pesosc1,biasc1,pesoscs,biascs = valores_obtenidos[0],valores_obtenidos[1],valores_obtenidos[2],valores_obtenidos[3]
  y_predict_list = []
  for entrada in x_test:
    salida_c1, output_cs = red_neuronal_multicapa(entrada, pesosc1,biasc1,pesoscs,biascs)
    y_predict_list.append(output_cs)
  return y_predict_list

# DETECCION DE VINO CON RNA

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv("wine.csv")
df.head()

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280_OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# =========== NORMALIZANDO =========== 
X = df.drop("class", axis=1)
Y = df["class"]

scaler = preprocessing.StandardScaler()
scaler.fit(X)
# =========== TRANSFORMANDO =========== 
X = scaler.transform(X)

In [ ]:
# =========== SEPARACION DE DATOS =========== 
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

## modelamiento

In [ ]:
# =========== FIT =========== 
pesosc1,biasc1,pesoscs,biascs = fit_rna(X_train,y_train,0.001,28,3,0.005,200)
pesos_bias_optimos = [pesosc1,biasc1,pesoscs,biascs] # recuperar los pesos y bias optimos

In [ ]:
# =========== PREDICT =========== 
y_pred_ = predict_rna(X_test, pesos_bias_optimos)     # predecir

In [ ]:
# =========== ajustar salida ================ 
def ajuste(valores_pred):
  result_pred = []
  for vector in valores_pred:
    indice = np.argmax(vector)
    result_pred.append(indice+1)
  y_pred = pd.Series(result_pred)
  return y_pred

In [ ]:
#y_pred = pd.Series(ajuste(y_pred)) # ajustar valores de salida

In [ ]:
y_pred = ajuste(y_pred_)

## metricas

In [ ]:
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

           1       0.93      1.00      0.97        14
           2       1.00      0.93      0.96        14
           3       1.00      1.00      1.00         8

    accuracy                           0.97        36
   macro avg       0.98      0.98      0.98        36
weighted avg       0.97      0.97      0.97        36



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score, f1_score

valores_pred = predict_rna(X_train, pesos_bias_optimos)
y_train_pred = pd.Series(ajuste(valores_pred))

# verificando aprendizaje
print("Aprendizaje - Entrenamiento")
print(balanced_accuracy_score(y_train, y_train_pred))
print(f1_score(y_train, y_train_pred, average="weighted"))
print('\n')
#verificando generalización
print("Prueba - Generalización")
print(balanced_accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="weighted"))

Aprendizaje - Entrenamiento
0.980896686159844
0.9788232288959365


Prueba - Generalización
0.9761904761904763
0.9721867461331064


## prediccion

In [ ]:
def leer_datos():
  features = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash',
       'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
       'Proanthocyanins', 'Color intensity', 'Hue', 'OD280_OD315', 'Proline']
  data_input = []
  for feature in features:
    value = float(input(f"insert value of {feature}: "))
    data_input.append(value)
  return data_input

In [ ]:
entrada = leer_datos()

insert value of Alcohol: 15
insert value of Malic acid: 1.8
insert value of Ash: 2.55
insert value of Alcalinity of ash: 14
insert value of Magnesium: 120
insert value of Total phenols: 2.70
insert value of Flavanoids: 3.01
insert value of Nonflavanoid phenols: 0.33
insert value of Proanthocyanins: 2.35
insert value of Color intensity: 5.55
insert value of Hue: 1.09
insert value of OD280_OD315: 3.55
insert value of Proline: 1060


In [ ]:
entrada_df = np.array([entrada]) # convertir a array de numpy y lista de listas
# NORMALIZANDO LOS DATOS DE ENTRADA
entrada_df = scaler.transform(entrada_df)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
entrada_df

array([[ 2.46976656, -0.48146012,  0.67069345, -1.65004916,  1.42241183,
         0.64876424,  0.98462062, -0.25667281,  1.33000903,  0.21278555,
         0.58154421,  1.32531572,  0.99708646]])

In [ ]:
result = np.argmax(predict_rna(entrada_df, pesos_bias_optimos))
if result == 0:
  print("CALIDAD BAJA")
elif result == 1 :
  print("CALIDAD REGULAR")
else:
  print("CALIDAD ALTA")

CALIDAD BAJA
